In [ ]:
#### The code is from Irene Yi-Ju Su ####
!pip3 install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from unsloth import FastLanguageModel
import numpy as np
import torch
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model_name = "unsloth/Llama-3.2-1B-Instruct"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/TEST")
import pandas as pd
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings('ignore')

df_QA = pd.read_csv("QA_pairs.csv")

Mounted at /content/drive


In [ ]:
max_seq_length = 512
dtype = None
load_in_4bit = True

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
model = FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.11.11: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
def stream_answer(question, model=model, tokenizer=tokenizer):
    messages = [
        {"role": "user", "content": question},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        return_tensors = "pt",
    ).to("cuda")
    attention_mask = torch.ones_like(inputs).to("cuda")
    input_length = inputs.shape[1]
    generated_ids = model.generate(
        input_ids = inputs,
        attention_mask = attention_mask,
        max_new_tokens = 512,
        use_cache = True,
    )
    answer = tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0]
    return answer
questions, answers = [], []
for i in range(len(df_QA)):
    question = df_QA["Question"][i]
    questions.append(question)
    answer = stream_answer(question)
    answers.append(answer)

In [ ]:
class SimplifiedSummarizer:
    def __init__(self, model_name="facebook/bart-large-cnn"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    def flatten_text(self, text: str) -> str:
        text = re.sub(r'\*\*', '', text)
        lines = []
        for line in text.split('\n'):
            line = line.strip()
            if not line:
                continue
            if re.match(r'^\d+\.', line):
                line = re.sub(r'^\d+\.\s*', '', line)
            lines.append(line)
        flattened = ' '.join(lines)
        flattened = re.sub(r'\s+', ' ', flattened)
        return flattened.strip()
    def generate_summary(self, text: str, max_length: int = 150) -> str:
        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            max_length=1024,
            truncation=True
        )
        outputs = self.model.generate(
            **inputs,
            max_length=max_length,
            min_length=30,
            num_beams=4,
            length_penalty=2.0,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
class LlamaSummarizer:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer

    def flatten_text(self, text: str) -> str:
        text = re.sub(r'\*\*', '', text)
        lines = []
        for line in text.split('\n'):
            line = line.strip()
            if not line:
                continue
            if re.match(r'^\d+\.', line):
                line = re.sub(r'^\d+\.\s*', '', line)
            lines.append(line)
        flattened = ' '.join(lines)
        flattened = re.sub(r'\s+', ' ', flattened)
        return flattened.strip()

    def generate_summary(self, text: str) -> str:
        flattened_text = self.flatten_text(text)
        prompt = f"Please summarize the following text clearly and concisely:\n\n{flattened_text}"

        messages = [
            {"role": "user", "content": prompt},
        ]
        inputs = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            return_tensors="pt",
        ).to("cuda")

        attention_mask = torch.ones_like(inputs).to("cuda")
        input_length = inputs.shape[1]

        generated_ids = self.model.generate(
            input_ids=inputs,
            attention_mask=attention_mask,
            max_new_tokens=150,  # Adjust this for summary length
            min_new_tokens=30,
            use_cache=True,
        )

        summary = self.tokenizer.batch_decode(
            generated_ids[:, input_length:],
            skip_special_tokens=True
        )[0]

        return summary

In [ ]:
answers_simple = [] # get simple answer for 100 QA by facebook/bart-large-cnn
for i in range(len(answers)):
    summarizer = SimplifiedSummarizer()
    flattened = summarizer.flatten_text(answers[i])
    summary = summarizer.generate_summary(flattened)
    answers_simple.append(summary)

In [ ]:
def clean_llama_summary(text): #remove the annoying assistant\n\nHere is a clear and concise summary of the text:\n\n and other variations
   text = text.replace("<userStyle>Normal</userStyle>", "")
   text = text.replace("assistent", "")
   text = text.replace("assistant", "")
   text = text.replace("assistant,", "")
   colon_count = text.count(":")
   if colon_count >= 1:
       return text.split(":", 1)[1].strip()
   else:
       return text.strip()

In [ ]:
answers_simple2 = [] # get simple answer for 100 QA by llama3.2-1B-instruct
for i in range(len(answers)):
    summarizer = LlamaSummarizer(model = model, tokenizer = tokenizer)
    flattened = summarizer.flatten_text(answers[i])
    summary = summarizer.generate_summary(flattened)
    answers_simple2.append(summary)

In [ ]:
df_QA2 = pd.DataFrame({"questions": questions, "answers": answers, "answers_simple1_bart_large_cnn": answers_simple, "answers_simple2_llama": answers_simple2})
df_QA2["answers_simple2_llama"] = df_QA2["answers_simple2_llama"].apply(clean_llama_summary)
df_QA2.to_csv("QA_pairs_simple.csv", index = False)

In [ ]:
df_QA2

,questions,answers,answers_simple1_bart_large_cnn,answers_simple2_llama
0,"Why is the ""Black Death"" mentioned so often in western history but not Chinese history even though it originated there?","assistant\n\nThe ""Black Death"" is a historical phenomenon that originated in Europe and spread globally, but it has been largely forgotten in Chinese history. There are several reasons for this:\n\n1. **Limited European records**: Although the plague originated in the Middle East and was introduced to Europe through the Silk Road, European records of the plague are limited and often biased. Many European cities and towns were already experiencing outbreaks of the plague before the Black Death, and the records from these areas were not well-documented or widely disseminated.\n2. **Chinese cultural sensitivity**: In Chinese culture, the plague is often seen as a natural disaster or a punishment from heaven. This perception has led to a lack of documentation and a lack of attention to the plague in historical records.\n3. **Lack of European influence**: Unlike European history, which has been extensively documented and analyzed, Chinese history has been subject to various interpretations and biases. The Chinese perspective on the plague has often been shaped by Confucianism, Taoism, and other philosophical traditions that emphasize the importance of social hierarchy and the concept of ""yin-yang.""\n4. **Limited archaeological evidence**: While there is archaeological evidence of the plague in Europe, there is little similar evidence in China. This lack of evidence has contributed to the perception that the plague was not a significant historical event in China.\n5. **Historical amnesia**: The Chinese have a long history of forgetting and amnesia, which is a result of the country's complex social and cultural structures. This phenomenon is often referred to as "" Chinese amnesia.""\n\nHowever, it is worth noting that there is evidence of a significant outbreak of the plague in China during the 14th century. The Chinese historian Su Shi (1036-1100) wrote about the plague in his book ""Shiji,"" which is a comprehensive history of China. Additionally, there are records of outbreaks of the plague in China during the Song Dynasty (960-1279 AD).\n\nIn recent years, there has been a growing interest in the history of the plague in China, and some historians have attempted to reconstruct the plague's impact on Chinese society. However, much more work needs to be done to fully understand the plague's historical significance in China.\n\nIn summary, the ""Black Death"" is often mentioned in Western history due to limited European records, cultural sensitivity, and a lack of European influence. However, the plague has a significant historical presence in Chinese culture, and there is evidence of outbreaks in China during the 14th century.","The ""Black Death"" originated in the Middle East and was introduced to Europe through the Silk Road. European records of the plague are limited and often biased. In Chinese culture, the plague is often seen as a natural disaster or a punishment from heaven.","The ""Black Death"" is a historical phenomenon that occurred in Europe and spread globally, but its significance in Chinese history is largely forgotten. Limited European records, cultural sensitivity, and a lack of European influence have contributed to this lack of documentation. Archaeological evidence is also scarce in China. However, Chinese historians have written about the plague in their books, and there are records of outbreaks in China during the 14th century. Despite these efforts, much more work needs to be done to fully understand the plague's historical significance in China."
1,How historically accurate is the show The Last Kingdom on Netflix?,"assistant\n\nThe Last Kingdom, a historical drama series based on the novels by Bernard Cornwell, is set in 9th-century England. While the show takes creative liberties with the past, it does dra